In [1]:
import re
from collections import Counter
import textdistance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from string import punctuation
punctuation += "«»—…“”"
import unittest
import logging

In [2]:
#словарь "правильных" слов (из корпуса с абстрактами из википедии - для русского) + векторное представление
class Vocabulary:
    def __init__(self, file = 'data/wiki_data.txt'):
        corpus = open(file, encoding='utf8').read()
        #словарь + количество вхождений
        self.vocab = Counter(re.findall('\w+', corpus.lower()))
        
    def vectorize(self):
        self.word2id = list(self.vocab.keys())
        self.id2word = {i:word for i, word in enumerate(self.vocab)}
        self.vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
        self.X = self.vec.fit_transform(self.vocab)

In [3]:
class WordCorrection:
    def __init__(self, word, voc):
        self.voc = voc
        self.voc.vectorize()
        self.N = sum(voc.vocab.values())
        self.word = word.lower()
    
    def get_closest_match_vec(self, topn=20):
        #сначала находим ближайших кандидатов на исправление через косинусную близость
        v = self.voc.vec.transform([self.word])
        similarities = cosine_distances(v, self.voc.X)[0]
        topn = similarities.argsort()[:topn] 
        return [(self.voc.id2word[top], similarities[top]) for top in topn]
    
    def get_closest_match_with_metric(self, lookup,topn=20, metric=textdistance.levenshtein):
        #среди кандидатов находим слова с минимальным расстоянием Левенштейна
        similarities = Counter() 
        for w in lookup:
            similarities[w] = metric.normalized_similarity(self.word, w) 

        return similarities.most_common(topn)

    def P(self, w):
        #вероятность слова
        return self.voc.vocab[w] / self.N
    
    def predict_mistaken(self):
        #считаем слово правильным (без опечаток), если оно есть в словаре
        return 0 if self.word in self.vocab else 1
    
    def get_closest_hybrid_match(self, topn=3, metric=textdistance.damerau_levenshtein):
        #находим близкие слова через косинусную близость
        candidates = self.get_closest_match_vec(topn*4) 
        lookup = [cand[0] for cand in candidates]
        #считаем расстояние Левенштейна для кандидатов 
        closest = self.get_closest_match_with_metric(lookup, metric=metric)
        dist = closest[0][1]
        closest_res = Counter() #словарь с кандидатами с одинаковым расстоянием редактирования (лучшим)
        closest_res[closest[0]] = self.P(closest[0][0])
        i = 1
        while i < len(closest) and closest[i][1] == dist:
            closest_res[closest[i]] = self.P(closest[i][0]) #каждому кандидату присваиваем вероятность
            i += 1
        self.correction = closest_res.most_common(1)[0][0]
        return self.correction[0]

In [4]:
class Tokenize:
    #небольшой класс для токенизации текста
    def __init__(self, text):
        self.text = text.lower()
    def text2tokens(self):
        sentences = sent_tokenize(self.text, language='russian')
        self.tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

                

In [5]:
class TokenizeEn(Tokenize):
    #такой же но для английского
    def text2tokens(self):
        sentences = sent_tokenize(self.text, language='english')
        self.tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

In [7]:
class Logger():
    log = []    
    # Переопределим оператор выделения объекта.
    def __new__(cls, *args, **kwargs):
        # Проверим есть ли у класса этот атрибут.
        if not hasattr(cls, '_logger'):
            # Просим родительский класс создать объект типа, переданного в cls, то есть Logger.
            cls._logger = super(Logger, cls).__new__(cls, *args, **kwargs)
        return cls._logger
        
    def logIt(self, s: str):
        Logger.log.append(s)

## фасад

In [8]:
 class SpellCheck:
    def __init__(self, text, lang = 'ru'):
        self.lang = lang
        if lang == 'ru':
            self.voc = Vocabulary()
            self.tokenize = Tokenize(text = text)
        elif lang == 'en':
            self.voc = Vocabulary(file = 'data/wiki_data_en.txt')
            self.tokenize = TokenizeEn(text = text)
        self.tokenize.text2tokens()
        self.tokens = self.tokenize.tokenized_sentences
        self.log = Logger()
    
    def text(self, text):
        #менять текст не обновляя словарь
        if self.lang == 'ru':
            self.tokenize = Tokenize(text = text)
        elif self.lang == 'en':
            self.tokenize = TokenizeEn(text = text)
        self.tokenize.text2tokens()
        self.tokens = self.tokenize.tokenized_sentences
        
    def predict_mistaken(self, word):
        #считаем слово правильным (без опечаток), если оно есть в словаре
        return 0 if word in self.voc.vocab else 1
        
    def analyze(self):
        self.result = ''
        for s in self.tokens:
            for word in s:
                if word not in punctuation and self.predict_mistaken(word) == 1:
                    try:
                        corr = WordCorrection(word, self.voc)
                        corr.get_closest_hybrid_match()
                        word = corr.correction[0]
                    except:
                        #если не нашлось совсем никакого слова > оно не заменяется + записывается в логи
                        self.log.logIt(word)
                        word = word
                self.result = self.result + word + ' '
            self.result+= ' '
        self.result = re.sub(r' ([\.,:;!?])', r'\1', self.result, flags=re.DOTALL)
        self.result = self.result.rstrip()
        return self.result

In [9]:
tst = SpellCheck('вобще, это была страная идея')

In [10]:
tst.analyze()

'вообще, это была странная идея'

In [14]:
tst.log.log

[]

In [12]:
tst.text('сонце светит')
tst.analyze()

'солнце светит'

In [95]:
w = WordCorrection('увидл', v)
w.get_closest_hybrid_match()
w.correction

('увидел', 0.8333333333333334)

In [15]:
tst_en = SpellCheck('Hwose wooods these are I thinc I know.', lang='en')

In [16]:
tst_en.analyze()

'whose woods these are i think i know.'

## тестирование

In [19]:
class TestSpelling(unittest.TestCase):
        
    def test_sun(self):
        a = SpellCheck('сонце светит')
        self.assertEqual(a.analyze(), 'солнце светит')
        
    def test_idea(self):
        b = SpellCheck('сонце светит')
        b.text('вобще, это была страная идея')
        self.assertEqual(b.analyze(), 'вообще, это была странная идея')
        
    def test_long_words(self):
        c = SpellCheck('this is a loooooooong wooooooord', lang='en')
        self.assertEqual(c.analyze(), 'this is a long word')
        
    def test_frost(self):
        c = SpellCheck('Hwose wooods these are I thinc I know.', lang='en')
        self.assertEqual(c.analyze(), 'whose woods these are i think i know.')
    
    
        

In [20]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_frost (__main__.TestSpelling) ... /Users/ignatenkodarja/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/wiki_data_en.txt' mode='r' encoding='utf8'>
  after removing the cwd from sys.path.
ok
test_idea (__main__.TestSpelling) ... /Users/ignatenkodarja/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='data/wiki_data.txt' mode='r' encoding='utf8'>
  after removing the cwd from sys.path.
ok
test_long_words (__main__.TestSpelling) ... FAIL
test_sun (__main__.TestSpelling) ... ok

FAIL: test_long_words (__main__.TestSpelling)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/48/0qdhk_tx72z356j9sh2k4dcm0000gn/T/ipykernel_1152/278204175.py", line 14, in test_long_words
    self.assertEqual(c.analyze(), 'this is a long word')
AssertionError: 'this is a ooooooo